In [2]:
import random
from datetime import datetime, timedelta
from transitions import Machine

# Define a function to calculate Manhattan distance
def calculate_travel_time(zone1, zone2):
    return abs(zone1[0] - zone2[0]) + abs(zone1[1] - zone2[1])

class Zone:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.name = f"Zone ({x},{y})"
        self.traffic_state = "normal_traffic"  # Simplified traffic management

class Driver:
    states = ['idle', 'en_route', 'waiting_for_rider', 'in_ride', 'ride_completed']

    def __init__(self, name, zone):
        self.name = name
        self.zone = zone
        self.time_to_next_zone = 0
        self.current_rider = None
        self.current_request = None
        self.machine = Machine(model=self, states=Driver.states, initial='idle')
        self.machine.add_transition('accept_ride', 'idle', 'en_route')
        self.machine.add_transition('arrive_at_pickup', 'en_route', 'waiting_for_rider')
        self.machine.add_transition('pick_up_rider', 'waiting_for_rider', 'in_ride')
        self.machine.add_transition('complete_ride', 'in_ride', 'ride_completed')
        self.machine.add_transition('reset', '*', 'idle')

    def __str__(self):
        return f"{self.name} (State: {self.state}, Zone: {self.zone.name}, Time to next zone: {self.time_to_next_zone})"

class Rider:
    states = ['waiting', 'matched', 'in_ride', 'ride_completed', 'ride_canceled']

    def __init__(self, name, zone):
        self.name = name
        self.zone = zone
        self.estimated_wait_time = 0
        self.machine = Machine(model=self, states=Rider.states, initial='waiting')
        self.machine.add_transition('match_with_driver', 'waiting', 'matched')
        self.machine.add_transition('start_ride', 'matched', 'in_ride')
        self.machine.add_transition('complete_ride', 'in_ride', 'ride_completed')
        self.machine.add_transition('cancel_ride', '*', 'ride_canceled')

    def __str__(self):
        return f"{self.name} (State: {self.state}, Zone: {self.zone.name}, Estimated wait time: {self.estimated_wait_time})"

class RideRequest:
    states = ['requested', 'accepted', 'in_progress', 'completed', 'canceled']

    def __init__(self, rider, origin_zone, destination_zone):
        self.rider = rider
        self.origin_zone = origin_zone
        self.destination_zone = destination_zone
        self.metadata = self.generate_random_metadata()
        self.machine = Machine(model=self, states=RideRequest.states, initial='requested')
        self.machine.add_transition('accept', 'requested', 'accepted')
        self.machine.add_transition('start', 'accepted', 'in_progress')
        self.machine.add_transition('complete', 'in_progress', 'completed')
        self.machine.add_transition('cancel', '*', 'canceled')

    def generate_random_metadata(self):
        distance = calculate_travel_time((self.origin_zone.x, self.origin_zone.y), (self.destination_zone.x, self.destination_zone.y))
        duration = distance  # 1 unit per zone
        return {
            "distance_km": distance,
            "duration_min": duration,
            "pickup_time": None,
            "dropoff_time": None,
            "cost_usd": round(random.uniform(5, 50), 2),
            "driver_rating": round(random.uniform(4.0, 5.0), 2),
            "rider_rating": round(random.uniform(4.0, 5.0), 2),
            "ride_type": random.choice(["economy", "premium", "pool"]),
            "traffic_conditions": random.choice(["normal", "heavy", "light"]),
            "weather_conditions": random.choice(["clear", "rainy", "stormy", "snowy"]),
        }

    def __str__(self):
        return f"RideRequest for {self.rider.name} (Origin: {self.origin_zone.name}, Destination: {self.destination_zone.name}, State: {self.state}, Metadata: {self.metadata})"

class StateTracker:
    def __init__(self, drivers, riders, ride_requests):
        self.drivers = drivers
        self.riders = riders
        self.ride_requests = ride_requests

    def print_states(self, label):
        print(f"\n{label}")
        for rider in self.riders:
            print(rider)
        for driver in self.drivers:
            print(driver)
        for request in self.ride_requests:
            print(request)

# Create the 10x10 grid of zones
zones = {(x, y): Zone(x, y) for x in range(1, 11) for y in range(1, 11)}

# Initialize 2 riders and 3 drivers
riders = [Rider(f'Rider {i}', random.choice(list(zones.values()))) for i in range(2)]
drivers = [Driver(f'Driver {i}', random.choice(list(zones.values()))) for i in range(3)]
ride_requests = [RideRequest(random.choice(riders), random.choice(list(zones.values())), random.choice(list(zones.values()))) for _ in range(2)]

# Create the state tracker
state_tracker = StateTracker(drivers, riders, ride_requests)

# Function to find the nearest available driver
def find_nearest_driver(rider_zone):
    nearest_driver = None
    min_travel_time = float('inf')
    for driver in drivers:
        if driver.state == 'idle':
            travel_time = calculate_travel_time((driver.zone.x, driver.zone.y), (rider_zone.x, rider_zone.y))
            if travel_time < min_travel_time:
                min_travel_time = travel_time
                nearest_driver = driver
    return nearest_driver, min_travel_time

# Simulation workflow
def simulate_rideshare():
    time_unit = 0

    # Print initial ride requests
    state_tracker.print_states("Initial ride requests")

    while any(rider.state != 'ride_completed' for rider in riders):
        print(f"\nTime Unit: {time_unit}")

        # Print states before updates
        state_tracker.print_states("Before updates")

        for rider in riders:
            if rider.state == 'waiting':
                # Find the nearest driver
                nearest_driver, travel_time = find_nearest_driver(rider.zone)
                if nearest_driver:
                    print(f"{nearest_driver.name} will pick up {rider.name} from {rider.zone.name}. Travel time: {travel_time}")
                    nearest_driver.accept_ride()
                    nearest_driver.time_to_next_zone = travel_time
                    nearest_driver.current_rider = rider
                    nearest_driver.current_request = next((request for request in ride_requests if request.rider == rider), None)
                    if nearest_driver.current_request:
                        nearest_driver.arrive_at_pickup()
                        nearest_driver.current_request.metadata["pickup_time"] = datetime.now() + timedelta(minutes=time_unit)
                        nearest_driver.current_request.accept()
                        rider.match_with_driver()
                        rider.estimated_wait_time = travel_time
                else:
                    rider.estimated_wait_time += 1
                    print(f"No driver available for {rider.name}. Estimated wait time: {rider.estimated_wait_time} units.")

        for driver in drivers:
            if driver.time_to_next_zone > 0:
                driver.time_to_next_zone -= 1
                print(f"{driver.name} is traveling. Time to next zone: {driver.time_to_next_zone}")
            elif driver.time_to_next_zone == 0:
                if driver.state == 'en_route':
                    driver.arrive_at_pickup()
                    driver.zone = driver.current_rider.zone
                    print(f"{driver.name} has arrived at the pickup zone for {driver.current_rider.name}.")
                elif driver.state == 'waiting_for_rider':
                    driver.pick_up_rider()
                    driver.current_request.start()
                    driver.current_rider.start_ride()
                    print(f"{driver.name} picked up {driver.current_rider.name} and is now in ride.")
                elif driver.state == 'in_ride':
                    driver.complete_ride()
                    driver.current_request.metadata["dropoff_time"] = datetime.now() + timedelta(minutes=time_unit)
                    driver.current_request.complete()
                    driver.current_rider.complete_ride()
                    print(f"{driver.name} completed the ride with {driver.current_rider.name}.")
                    driver.current_rider = None
                    driver.current_request = None
                    driver.reset()
                    print(f"{driver.name} is now idle.")

        # Print states after updates
        state_tracker.print_states("After updates")

        time_unit += 1

simulate_rideshare()



Initial ride requests
Rider 0 (State: waiting, Zone: Zone (7,7), Estimated wait time: 0)
Rider 1 (State: waiting, Zone: Zone (5,1), Estimated wait time: 0)
Driver 0 (State: idle, Zone: Zone (10,9), Time to next zone: 0)
Driver 1 (State: idle, Zone: Zone (5,10), Time to next zone: 0)
Driver 2 (State: idle, Zone: Zone (6,5), Time to next zone: 0)
RideRequest for Rider 1 (Origin: Zone (5,6), Destination: Zone (5,10), State: requested, Metadata: {'distance_km': 4, 'duration_min': 4, 'pickup_time': None, 'dropoff_time': None, 'cost_usd': 25.06, 'driver_rating': 4.91, 'rider_rating': 4.1, 'ride_type': 'pool', 'traffic_conditions': 'light', 'weather_conditions': 'rainy'})
RideRequest for Rider 0 (Origin: Zone (4,1), Destination: Zone (1,2), State: requested, Metadata: {'distance_km': 4, 'duration_min': 4, 'pickup_time': None, 'dropoff_time': None, 'cost_usd': 26.33, 'driver_rating': 4.37, 'rider_rating': 4.73, 'ride_type': 'economy', 'traffic_conditions': 'normal', 'weather_conditions': 'sto